In [15]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from tabulate import tabulate
from matplotlib import pyplot as plt
import scipy.stats as st
import statsmodels.api as sm
import seaborn as sns
import functions as fun
import OLS_Estimation as OLS


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


To datasæt. Det originale med varierende pris og et manipuleret sådan at ccp for en bil hvor $ms = 0$ er $\approx 0$

In [16]:
df = pd.read_csv('Dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
df_straf = df.copy()
df_straf = fun.straf_0ms(df_straf)


print(f'shape OG: {df.shape} \n shape straf: {df_straf.shape}')

c:\Users\Mikkel\OneDrive\Offentlig\Dokumenter\GitHub\Bachelor\functions.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = -df['Price'][i]*1000


shape OG: (1177, 14) 
 shape straf: (1177, 14)


# Manipulerer data til OLS estimation

## Creating dummies

In [17]:
df2 = df.copy()
print(f'1 df shape: {df.shape} df2 shape {df2.shape}')
df2 = pd.get_dummies(df2, columns=['Segment'], drop_first=True)
print(f'2 df shape: {df.shape} df2 shape {df2.shape}')
df2 = pd.get_dummies(df2, columns=['Year'], drop_first=True)
print(f'3 df shape: {df.shape} df2 shape {df2.shape}')
df2['China'] = (df2['Country'] == 'CH').astype(int)
print(f'4 df shape: {df.shape} df2 shape {df2.shape}')

1 df shape: (1177, 14) df2 shape (1177, 14)
2 df shape: (1177, 14) df2 shape (1177, 20)
3 df shape: (1177, 14) df2 shape (1177, 29)
4 df shape: (1177, 14) df2 shape (1177, 30)


## Creating log market share

In [18]:
df2['Market share'] = df2['Market share'].replace(0, 1)
df2['log_market_share'] = np.log(df2['Market share']) 
df2['log_market_share'].head(5)
print(f'df shape: {df.shape} df2 shape {df2.shape}')
#print(f'{df2.columns} \n {df.columns}')

df shape: (1177, 14) df2 shape (1177, 31)


## Independent OLS

In [19]:
#Fjernede vi modellerne som gav fejl?
data = df2

print(f'data shape: {data.shape} df shape {df.shape}')

#data format

N = data['ID'].unique().size
T = df['Year'].unique().size
print(f'N: {N} T: {T} N*T: {N*T}')
print(f'Data has N={N} and T={T} data is = {data.shape[0]}, {N*T}')

variables = ['Price', 'HP', 'Chargetime', 'Range','Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J', 'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023', 'China']
xnames = ['Cons'] + variables


#Declare variables
#y = data["log_market_share"].values.reshape((N*T,1))
X = np.hstack([np.ones((N*T, 1))] + [data[var].values.reshape((N*T, 1)) for var in variables])
print(f'Variables: {len(variables)}')


#Estimate with OLS

#results = fun.est_OLS(y, X, xnames)
#params = results.params
#print(f'params: {params}')
#fun.logit(y, X, xnames)


data shape: (1177, 31) df shape (1177, 14)
N: 107 T: 11 N*T: 1177
Data has N=107 and T=11 data is = 1177, 1177
Variables: 22


In [21]:
#X skal være [i:i+1,2:] 2: for at fjerne consten og pris
#beta skal være [2:] for at fjerne konstant og pris-koefficienten og reshapes for at kunne lave matrixmultiplikaiton. 

coefficients = OLS.get_coefficients_OLS(pd.read_csv('Dataset.csv'))
print(f'coefficients:\n{coefficients[:10]}\nshape: {coefficients.shape}')

coefficients_labels = xnames[1:] #Fjerne konstanten

model_labels = data['Model'].unique()
j = 0
for i in model_labels:
    j = j
    #print(f'Car {j}: {i}')
    j = j+1  
print(f'models: {model_labels[:3]}')

repeated_models = np.repeat(model_labels, T)

alpha = coefficients[1]
beta = coefficients

print(f'N: {N} T: {T} N*T: {N*T}')
p_j = df_straf['Price'].values.reshape((N*T,1))
print(f'\n \n p_j shape: {p_j.shape}') 

coefficients:
const        -3.628008e+00
Price        -1.869419e-08
Range         8.307278e-03
HP           -1.842563e-03
Chargetime   -3.447909e-02
Segment_B    -3.216900e-01
Segment_C     3.049201e-01
Segment_D    -7.320388e-02
Segment_E    -1.514931e+00
Segment_F    -1.772606e+00
dtype: float64
shape: (23,)
models: ['U5' 'U6' 'e-tron']
N: 107 T: 11 N*T: 1177

 
 p_j shape: (1177, 1)


In [22]:
#ccp_list = fun.ccp(alpha, beta, X, p_j)
share_j = []
exp_share_j = []
for j in range(len(p_j)):
    s = alpha*p_j[j] + X[j:j+1,2:]@beta[2:]
    share_j.append(s)
print(f'share_j: {len(share_j)}')
    
for j in range (len(share_j)):
    exp_share_j.append(np.exp(share_j[j]))
print(f'exp_share_j: {np.sum(exp_share_j)}')

share_j: 1177
exp_share_j: 0.22153183755592828


In [23]:
def exp_delta(alpha, beta, X, p_j):
    share_j = []
    exp_share_j = []
    for j in range(len(p_j)):
        s = alpha*p_j[j] + X[j:j+1,2:]@beta[2:]
        share_j.append(s)
        #print(f'share_j: {share_j[7:11]}')
        
    for j in range (len(share_j)):
        exp_share_j.append(np.exp(share_j[j]))
    return exp_share_j

def ccp(alpha, beta, X, p_j):
    ccp_list = [] 
    exp_delta_list = exp_delta(alpha, beta, X, p_j)
    sum_exp = np.sum(exp_delta_list)

ccp_list = ccp(alpha, beta, X, p_j)
print(ccp_list)

None


In [24]:
m = 3

In [25]:
fun.probability_ratio(ccp_list[:m], repeated_models[:m], repeated_models[:m])

TypeError: 'NoneType' object is not subscriptable

In [ ]:
fun.marginal_effects(ccp_list[:m], repeated_models[:m], coefficients_labels[:m], coefficients[:m])

,Price,HP,Chargetime
U5,[[-5.279617962689559e-12]],[[3.732223223479503e-07]],NaN
U5,[[-5.505870383042135e-12]],[[3.892163685758361e-07]],NaN
U5,[[-5.263034730522306e-12]],[[3.720500344162787e-07]],NaN


In [ ]:
fun.cross_marginal_effects(ccp_list[:m], coefficients[:m])

(3, 3, 2)


array([[[ 6.02006377e-15, -4.25586092e-10],
        [ 6.27804720e-15, -4.43824131e-10],
        [ 6.00115480e-15, -4.24249329e-10]],

       [[ 6.27804720e-15, -4.43824131e-10],
        [ 6.54708625e-15, -4.62843743e-10],
        [ 6.25832791e-15, -4.42430084e-10]],

       [[ 6.00115480e-15, -4.24249329e-10],
        [ 6.25832791e-15, -4.42430084e-10],
        [ 5.98230523e-15, -4.22916766e-10]]])

In [ ]:
fun.elasticity(ccp_list[:m], repeated_models[:m], coefficients_labels[:m], coefficients[:m], X[:m])

elasticity shape: 
(3, 3)


,Price,HP,Chargetime
U5,[[1.533550854063779]],[[0.0658690222799449]],NaN
U5,[[1.5334759185806208]],[[0.06586580365371113]],NaN
U5,[[1.5335563464802262]],[[0.06586925819002273]],NaN


In [ ]:
fun.cross_elasticity(ccp_list[:m], coefficients[:m], X[:m], model_labels[:m])

3
Change in : U5 
 [[[-1.74862537e-03 -7.51068954e-05]]

 [[-1.74862537e-03 -7.51068954e-05]]

 [[-1.74862537e-03 -7.51068954e-05]]]
Change in : U6 
 [[[-1.82356086e-03 -7.83255217e-05]]

 [[-1.82356086e-03 -7.83255217e-05]]

 [[-1.82356086e-03 -7.83255217e-05]]]
Change in : e-tron 
 [[[-1.74313296e-03 -7.48709854e-05]]

 [[-1.74313296e-03 -7.48709854e-05]]

 [[-1.74313296e-03 -7.48709854e-05]]]
